# Necessary Packages for Training Datasetes      

In [1]:
# Imports
import numpy as np
import warnings 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
import keras.preprocessing.image
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Average, Input, Concatenate, GlobalMaxPooling2D,Conv2D
from keras.models import Model


In [2]:

from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop

# Assign Image size and Training Rate

In [3]:
warnings.filterwarnings('ignore')
IMAGE_SIZE = (128,128)
INPUT_SHAPE = (128,128, 3)
TRAINING_RATE = 0.001

# Load Training and Testing Datasets to generate Model

In [4]:
#read the train and test dataset
train_datagen = ImageDataGenerator()
validate_datagen = ImageDataGenerator()

training_set = train_datagen.flow_from_directory(
            'Dataset/train',
            target_size = IMAGE_SIZE,
            class_mode = 'categorical',
            batch_size = 20)

validation_set = validate_datagen.flow_from_directory(
            'Dataset/val',
            target_size = IMAGE_SIZE,
            class_mode = 'categorical',
            batch_size = 20)

classes = {v: k for k, v in training_set.class_indices.items()}
classes

Found 3229 images belonging to 4 classes.
Found 807 images belonging to 4 classes.


{0: 'MildDemented',
 1: 'ModerateDemented',
 2: 'NonDemented',
 3: 'VeryMildDemented'}

# Initialize Training method and Training Rate

In [5]:
#Model
input_shape = INPUT_SHAPE
inputs = Input(input_shape)
input_tensor = Input(shape= (128,128,3))

#xception = Xception(include_top=False, input_shape = input_shape,)(inputs)
xception = Xception(include_top=False, input_shape = input_shape, weights = 'imagenet')(inputs)
res_net = ResNet50(input_tensor = input_tensor, include_top = False, weights = 'imagenet')(inputs)

#output
outputs = Concatenate(axis=-1)([GlobalAveragePooling2D()(xception), GlobalAveragePooling2D()(res_net)])
#outputs = Concatenate(axis=-1)([GlobalAveragePooling2D()(xception)])
outputs = Dropout(0.25)(outputs)
outputs = Dense(len(classes), activation='softmax')(outputs)

# Set Optimizer
opt1 = Adam(lr=TRAINING_RATE)
opt2 = SGD(lr=TRAINING_RATE)
opt3 =RMSprop(lr=TRAINING_RATE)

model = Model(inputs, outputs)
model.compile(optimizer=opt2,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 xception (Functional)          (None, 4, 4, 2048)   20861480    ['input_1[0][0]']                
                                                                                                  
 resnet50 (Functional)          (None, 4, 4, 2048)   23587712    ['input_1[0][0]']                
                                                                                                  
 global_average_pooling2d (Glob  (None, 2048)        0           ['xception[0][0]']           

# To Start Training Process

In [ ]:
# Training
Training_model = model.fit(training_set,
                    steps_per_epoch= len(training_set),
                    epochs=2,
                    validation_data= validation_set,
                    validation_steps =len(validation_set))

Epoch 1/2
162/162 [==============================] - 984s 6s/step - loss: 0.9879 - accuracy: 0.5649 - val_loss: 0.7482 - val_accuracy: 0.6853
Epoch 2/2
 47/162 [=======>......................] - ETA: 12:19 - loss: 0.5038 - accuracy: 0.7912

# Save Trainned Model

In [ ]:
model.save("Trained_Models/LastModel.h5")


# Visualize the Trainned model accuracy

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,5))
ax = ax.ravel()
for i, met in enumerate(['accuracy','loss']):
    ax[i].plot(Training_model.history[met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train'])
    
plt.show()

In [ ]:
plt.plot(Training_model.history['accuracy'])
plt.plot(Training_model.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss

plt.plot(Training_model.history['loss'])
plt.plot(Training_model.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()